# General Jupyter notebook Commands

In [0]:
# to display bin to show common operations
!ls /bin

In [0]:
%lsmagic

In [0]:
# it's called magics in Jupyter Notebooks
%%html
<marquee style='width: 30%; color: green;'><b>HEDW 2020!</b></marquee>

In [0]:
a=10
a

# Database Access

In [0]:
# Snowflake connector to Python
!pip install --upgrade snowflake-connector-python[pandas] --quiet

In [0]:
# Pandas - library for data manipulation, aggregation, ploting (Data Analysis Library built on python)
import pandas as pd
import snowflake.connector 

In [0]:
# connector & cursor objects for snowflake DB - connectors for major DBs are available
ctx = snowflake.connector.connect(
    user='[user]',
    password='[password]', 
    account='[account]',
    region='[region]',
    warehouse='[warehouse]',
    database='[database]',
    schema='[schema]',
    role='[role]'
    )
stu = ctx.cursor()

#Data Frame and Features

In [0]:
# Extracting data into a Data Frame
query ="select s.sat_math,  s.sat_verbal,  s.act_composite, s.sex, s.GPA  from student_score s"
#df = pd.read_sql(query, con = ctx)

# Read in data from Github
url= 'https://raw.githubusercontent.com/smathewsnd/ndhedw/master/HEDW_data.csv'
df = pd.read_csv(url,usecols=['sat_math','sat_verbal','act_composite','sex','GPA'])

df.head(5)

In [0]:
# Extract data into a list
try:
  stu.execute("select s.sat_math, s.sat_verbal, s.act_composite, s.sex, s.GPA from student_score s")
  rows = stu.fetchall()
finally:
  stu.close()
  ctx.close()

In [0]:
# Creating a different column names
usecols=['SAT MATH','SAT VERBAL','ACT COMPOSITE','GENDER','GPA']
df1= pd.DataFrame(rows, columns=usecols)

In [0]:
# displays first n rows
df1.tail(5)

In [0]:
# It returns the shape(#cols, #rows)
df1.shape

In [0]:
# to understand overall properties of dataset, including stats distribution
df1.describe()

In [0]:
# This parameter includes all the elements in the data frame
df1.describe(include='all')

In [0]:
# count of rows in this data frame
df1.count()

In [0]:
# average using group by
df.groupby(df1['GENDER']).mean()

In [0]:
# Count of rows Male and Female
print(df1[df1.GENDER == 'M'].count())
print(df1[df1.GENDER== 'F'].count())

In [0]:
# Histogram - Distribution of the data
hist = df1.hist(bins=20)

In [0]:
# Create Random Sampling of data
df1.sample(n=100, random_state = 2020)

In [0]:
# This is to create a sample data using 20% of the data
student_test = df.sample(frac = 0.2, random_state = 2020) 
student_test.head(5)

In [0]:
# loc is the operation of locating the element
student_test.loc[1001][0]

In [0]:
# iloc is the operation of locating the index
student_test.iloc[1][4]

In [0]:
# convert it to an array object and homogeneous data types
df.to_numpy()

#Data Storage

In [0]:
#Colab to mount the drive 
from google.colab import drive
drive.mount('/drive')

In [0]:
#Uploading the newly created data frame into a csv file in google drive
student_test.to_csv('/drive/My Drive/Colab Notebooks/HEDW_test_sample.csv', encoding='utf-8', index=False)

In [0]:
# this command reads the csv file that I uploaded in google drive
train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/HEDW_test_sample.csv')
print(train,10)

In [0]:
#github *.csv file download to create dataframe (web url to access the github file)
url= 'https://raw.githubusercontent.com/smathewsnd/ndhedw/master/HEDW_data.csv'
df = pd.read_csv(url,usecols=['sat_math','sat_verbal','act_composite','sex','GPA'])
print(df)

In [0]:
# upload *.csv file to S3 bucket - upload df to S3
import s3fs
s3 = s3fs.S3FileSystem(secret='[secret_key]', key='[key]')

# Use 'w' for py3, 'wb' for py2
with s3.open('[bit_bucket_name]/demo/HEDW_data.csv','w') as f:
    df.to_csv(f,sep=',',  index=False)

#Create Table and Load Data

In [0]:
#Using python script create, insert and select tables; insert S3 csv file; Snowflake_Stage - @DEMO_DATA File Format = DEMO_DB.PUBLIC.csv
try:
  stu.execute("create or replace table student_score(sat_math integer, sat_verbal integer, act_composite integer, sex varchar(1), GPA number(3,2));")
  stu.execute("COPY INTO student_score FROM @DEMO_DATA/demo/HEDW_data.csv FILE_FORMAT ='DEMO_DB.PUBLIC.csv'")
finally:
  stu.close()
  ctx.close()

#Exploratory Data Analysis

In [0]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# necessary to see plots in the notebook
%matplotlib inline

In [0]:
# print histogram of SAT math scores
sns.distplot(df['SAT_MATH'], kde = False, bins = 20)

In [0]:
# print scatter plot of SAT verbal vs. SAT math
sns.scatterplot(x = 'SAT_MATH', y = 'SAT_VERBAL', data = df)

In [0]:
# plot boxplot for SAT math scores, broken down by sex
sns.boxplot(x = 'SEX', y = 'SAT_MATH', hue = 'SEX', data = df)

In [0]:
# plot boxplot for GPA, broken down by sex
sns.boxplot(x = 'SEX', y = 'GPA', hue = 'SEX', data = df)

In [0]:
# create new random variable
np.random.seed(2020)
df['RANDOM_VAR'] = np.random.randint(low = 1, high = 100, size = 2000)

In [0]:
# look at the first five rows
df.head()

In [0]:
# print heatmap of correlation matrix for numeric fields
sns.heatmap(df.corr(), vmin = -.2)

#Simple Linear Regression

In [0]:
# plot simple linear regression line for GPA from SAT math
sns.lmplot(x = 'SAT_MATH', y = 'GPA', data = df, line_kws = {'color': 'green'})

In [0]:
# plot simple linear regression line for GPA from SAT math, broken down by sex
sns.lmplot(x = 'SAT_MATH', y = 'GPA', data = df, hue = 'SEX')

#Multiple Linear Regression

In [0]:
# import functions for linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [0]:
# obtain dummy variables for gender
df2 = pd.get_dummies(df, drop_first = True)

In [0]:
# look at first five rows of the original dataset for comparison
df.head()

In [0]:
# look at first five rows of the new dataset
df2.head()

In [0]:
# split data into predictors (X) and response (y)
X = df2[['SAT_MATH', 'SAT_VERBAL', 'ACT_COMPOSITE', 'RANDOM_VAR', 'SEX_M']]
y = df2['GPA']

# perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2020)

In [0]:
# perform multiple linear regression on training set
lm = LinearRegression()
lm.fit(X_train, y_train)

In [0]:
# print intercept of the resulting model
print(lm.intercept_)

In [0]:
# print coefficients of the resulting model
coeff_df = pd.DataFrame(lm.coef_, X_train.columns, columns = ['Coefficient'])
coeff_df

In [0]:
# make prediction on test set
predictions = lm.predict(X_test)

In [0]:
# plot predicted GPA vs. actual GPA for test set
fig, axes = plt.subplots()
fig = sns.scatterplot(y_test, predictions)
axes.set_xlabel('Actual GPA')
axes.set_ylabel('Predicted GPA')
axes.set_aspect(1)
plt.plot([0.0, 4.0], [0.0, 4.0], color = 'red')

In [0]:
# want to see p-values; need to use a different function
import statsmodels.api as sm

# fit multiple linear regression model again using different function
X2_train = sm.add_constant(X_train)
test2 = sm.OLS(y_train, X2_train)
test2 = test2.fit()

# print summary of resulting model
print(test2.summary())

In [0]:
# predict someone's GPA
lm.predict([[600, 600, 25, 50, 0]])